In [1]:
import sys
import mrob
import time
import os
from octreelib.grid import Grid, GridConfig, VisualizationConfig
from octreelib.octree import MultiPoseOctree, OctreeConfig
from typing import Tuple, List
import numpy as np
import open3d as o3d

sys.path.append("..")
if True:
    from slam.backend import BaregBackend, EigenFactorBackend
    from slam.pipeline import StaticPipeline, StaticPipelineRuntimeParameters
    from slam.segmenter import CAPESegmenter, RansacSegmenter
    from slam.subdivider import CountSubdivider, EigenValueSubdivider, SizeSubdivider
    from slam.utils import HiltiReader, KittiReader

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Benchmark configuration

In [2]:
SAMPLES_COUNT = 10

### Utils

In [3]:
def pretty_print(subdivide_ts: List[float], segmenters_ts: List[float], backend_ts: List[float]):
    def metrics(ts: List[float]):
        print(f"\tmin = {round(np.min(subdivide_ts), 3)}s")
        print(f"\tmax = {round(np.max(subdivide_ts), 3)}s")
        print(f"\tmean = {round(np.mean(subdivide_ts), 3)}s")
        print(f"\tstd = {round(np.std(subdivide_ts), 3)}s")
        
    subdivide_ts = np.array(subdivide_ts)
    segmenters_ts = np.array(segmenters_ts)
    backend_ts = np.array(backend_ts)
    
    print("Subdivide stage:")
    metrics(subdivide_ts)
    print("Segmenters stage:")
    metrics(segmenters_ts)
    print("Backend stage:")
    metrics(backend_ts)

## Pipeline

In [4]:
step = 3

def pipeline(point_clouds: List[o3d.geometry.PointCloud]) -> Tuple[float, float, float]:
    # Pipeline configuration
    # ----------------------
    subdividers = [
        SizeSubdivider(
            size=2,
        ),
    ]
    
    segmenters = [
        RansacSegmenter(
            threshold=0.005,
            initial_points=6,
            iterations=5000,
        ),
    ]
    
    backend = EigenFactorBackend(
        poses_number=step,
        iterations_number=5000,
    )

    grid = Grid(
        GridConfig(
            octree_type=MultiPoseOctree,
            octree_config=OctreeConfig(),
            grid_voxel_edge_length=4,
        )
    )
    # ----------------------
    
    middle_pose_number = 1
    
    grid.insert_points(
            middle_pose_number,
            point_clouds[middle_pose_number].points,
    )
    
    subdivide_start = time.perf_counter()
    grid.subdivide(subdividers)
    subdivide_end = time.perf_counter() - subdivide_start

    for pose_number, point_cloud in enumerate(point_clouds):
        if pose_number == middle_pose_number:
            continue
        grid.insert_points(pose_number, point_cloud.points)

    segmenters_start = time.perf_counter()
    for segmenter in segmenters:
        grid.map_leaf_points(segmenter)
    segmenters_end = time.perf_counter() - segmenters_start

    backend_start = time.perf_counter()
    backend.process(grid)
    backend_end = time.perf_counter() - backend_start

    return subdivide_end, segmenters_end, backend_end

# Hilti dataset evaluation

In [ ]:
hilti_dataset_path = "../evaluation/hilti"
hilti_clouds = os.path.join(hilti_dataset_path, "clouds")
hilti_poses = os.path.join(hilti_dataset_path, "poses")

def read_hilti_patch(start: int) -> List[o3d.geometry.PointCloud]:
    point_clouds = []
    for s in range(start, start + step):
        point_cloud_path = os.path.join(hilti_clouds, str(s) + ".pcd")
        pose_path = os.path.join(hilti_poses, str(s) + ".txt")
        point_cloud = HiltiReader.read_point_cloud(filename=point_cloud_path)
        pose = HiltiReader.read_pose(filename=pose_path)

        point_cloud.transform(pose)
        point_clouds.append(point_cloud)

    return point_clouds
    

for ind in range(0, 27, step):
    print(f"Patch {ind} -> {ind + step}; Samples count = {SAMPLES_COUNT}")
    subdivide_ts, segmenters_ts, backend_ts = [], [], []
    
    for sample in range(SAMPLES_COUNT):
        point_clouds = read_hilti_patch(ind)
        subdivide_t, segmenters_t, backend_t = pipeline(point_clouds)
        
        subdivide_ts.append(subdivide_t)
        segmenters_ts.append(segmenters_t)
        backend_ts.append(backend_t)

    pretty_print(subdivide_ts, segmenters_ts, backend_ts)

Patch 0 -> 3; Samples count = 10
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterations equals 0
Iterati

# Kitti dataset evaluation

In [ ]:
hilti_dataset_path = "../evaluation/kitti"
hilti_clouds = os.path.join(hilti_dataset_path, "clouds")
hilti_poses = os.path.join(hilti_dataset_path, "poses")

def read_kitti_patch(start: int) -> List[o3d.geometry.PointCloud]:
    point_clouds = []
    for s in range(start, start + step):
        point_cloud_path = os.path.join(hilti_clouds, "0" * (len(str(s))) + str(s) + ".bin")
        pose_path = os.path.join(hilti_poses, str(s) + ".txt")
        point_cloud = HiltiReader.read_point_cloud(filename=point_cloud_path)
        pose = HiltiReader.read_pose(filename=pose_path)

        point_cloud.transform(pose)
        point_clouds.append(point_cloud)

    return point_clouds

for ind in range(0, 15, step):
    print(f"Patch {ind} -> {ind + step}; Samples count = {SAMPLES_COUNT}")
    subdivide_ts, segmenters_ts, backend_ts = [], [], []
    
    for sample in range(SAMPLES_COUNT):
        point_clouds = read_kitti_patch(ind)
        subdivide_t, segmenters_t, backend_t = pipeline(point_clouds)
        
        subdivide_ts.append(subdivide_t)
        segmenters_ts.append(segmenters_t)
        backend_ts.append(backend_t)

    pretty_print(subdivide_ts, segmenters_ts, backend_ts)